In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df=pd.read_excel("1_bank_statement.xlsx",sheet_name="transaction_data")

In [4]:
df.columns

Index(['Date', 'Description', 'Chq./Ref.No.', 'Value Date', 'Debit', 'Credit',
       'Balance'],
      dtype='object')

In [5]:
df['Balance'] = df['Balance'].apply(lambda x:re.sub('[^0-9.]','',x))
df['Balance'] = df['Balance'].apply(np.float64)


# Credit/Debit

In [ ]:
df_new=df.filter(['Date','Description','Balance'])

In [ ]:
df_new["Transaction Type"]="Credit"
df_new["Transaction Amount"]=0

In [ ]:
credit=0
debit=0

In [ ]:
df_new.loc[6]

In [ ]:
for index,row in df_new.iterrows():
    if index>0:
        if row['Balance']<df_new.loc[index-1,'Balance']:
            debited_amount=df.loc[index-1,'Balance']-row['Balance']
            df_new.loc[index,"Transaction Type"]="Debit"
            df_new.loc[index,"Transaction Amount"]=debited_amount
        elif row['Balance']>df_new.loc[index-1,'Balance']:
            credited_amount=row['Balance']-df.loc[index-1,'Balance']
            df_new.loc[index,"Transaction Type"]="Credit"
            df_new.loc[index,"Transaction Amount"]=credited_amount


In [ ]:
df_new

In [ ]:
df_new.where(df_new["Transaction Type"]=="Credit")

In [ ]:
total_credited_amount=df_new.loc[df_new["Transaction Type"]=="Credit","Transaction Amount"].sum()

In [ ]:
total_debited_amount=df_new.loc[df_new["Transaction Type"]=="Debit","Transaction Amount"].sum()

# Date MAB

In [ ]:
from datetime import datetime

In [ ]:
df['date'] = df['Date'].apply(lambda x: datetime.strptime(x, "%d/%m/%y").date())


In [ ]:
type(df['date'][1])

In [ ]:
df[["year", "month", "day"]] = df["date"].apply(str).str.split("-", expand=True)
agg_date = df.groupby(['year','month','day']).nth(-1)

In [ ]:
df_new=agg_date.filter(['date','Balance'])

In [ ]:
all_days = pd.date_range(df_new.date.min(), df_new.date.max(), freq='D')

In [ ]:
df_date=df_new.set_index('date').reindex(all_days).fillna(method='ffill').rename_axis('date').reset_index()

In [ ]:
df_date.to_excel('file.xlsx')

In [ ]:
df_date

In [ ]:
df_date[["year", "month", "day"]] = df_date["date"].apply(str).str.split("-", expand=True)
agg_date = df_date.groupby(['year','month'])['Balance'].mean()

In [ ]:
agg_date

# transaction type

In [21]:
salary='sal|salary'
payment_gateway="abps|ecom|paytm|RAZPRAZORPAYSOFTWARE|billdesk|gpay"
auto_debit="nach|ach|emi|ecs|msi"
bank_charges="DEBIT CARD ANNUAL FEE|SMS CHARGES|INSPECTION CHGS|GST|DOCUMENTATION CHARGE|PROCESSING FEE|Charges|CHEQUE BOOK ISSUE CHARGES|CHRGS|RTN CHQ CHGS|tbms|AMB CHRG|debit interest captialised"
atm="atm|atw|cash|atm csw|nwd|owd|cwdr|ats|eaw|nfs|awb|vat|mat"
cheque="by clearing|by cheque|clg"
net_banking="neft|tpt|rtgs|imps|utr|rtgs|TO TRANSFER|OFT|TPFT"
card_transactions="pos|POSTXN|PRCR|medr|vps|prcr|pcd"
bank_credit='credit|credit interest captialised'
bill_pay='bpay|bil|billpay'
loan='disburse'
demand_draft='dd/cc|dd'

In [6]:
df_new=df.filter(['Date','Description','Balance'])

In [7]:
df_new['Transaction Type']="Others"

In [8]:
df_new['Description']=df_new['Description'].apply(lambda x:re.sub('[^a-zA-Z]',' ',x))

In [22]:
df_new.loc[df_new['Description'].str.contains(salary,flags=re.IGNORECASE)==True,'Transaction Type']='Salary'
df_new.loc[df_new['Description'].str.contains(payment_gateway,flags=re.IGNORECASE)==True,'Transaction Type']='Payment Gateway'
df_new.loc[df_new['Description'].str.contains(auto_debit,flags=re.IGNORECASE)==True,'Transaction Type']='Auto Debit'
df_new.loc[df_new['Description'].str.contains(atm,flags=re.IGNORECASE)==True,'Transaction Type']='ATM'
df_new.loc[df_new['Description'].str.contains(cheque,flags=re.IGNORECASE)==True,'Transaction Type']='Cheque Transaction'
df_new.loc[df_new['Description'].str.contains(net_banking,flags=re.IGNORECASE)==True,'Transaction Type']='Net Banking'
df_new.loc[df_new['Description'].str.contains(card_transactions,flags=re.IGNORECASE)==True,'Transaction Type']='Card Transactions'
df_new.loc[df_new['Description'].str.contains(bank_charges,flags=re.IGNORECASE)==True,'Transaction Type']='Bank Charges'
df_new.loc[df_new['Description'].str.contains(bank_credit,flags=re.IGNORECASE)==True,'Transaction Type']='Bank Credit'
df_new.loc[df_new['Description'].str.contains(bill_pay,flags=re.IGNORECASE)==True,'Transaction Type']='Bill Pay'
df_new.loc[df_new['Description'].str.contains(loan,flags=re.IGNORECASE)==True,'Transaction Type']='Loan'


In [23]:
df_new.to_excel('Transaction Type.xlsx',engine='xlsxwriter')